In [1]:
import numpy as np
from tqdm import tqdm 
import os,scipy.misc
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import RMSprop
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.preprocessing.image import ImageDataGenerator


import warnings
warnings.filterwarnings('ignore')

os.environ['KERAS_BACKEND']='tensorflow' 
os.environ['TENSORFLOW_FLAGS']='floatX=float32,device=gpu'

Using TensorFlow backend.


## Build Model

In [2]:
conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)


def DCGAN_D(isize, nc, ndf):
    inputs = Input(shape=(isize, isize, nc))
    x = ZeroPadding2D()(inputs)
    x = Conv2D(ndf, kernel_size=4, strides=2, use_bias=False, kernel_initializer=conv_init)(x)
    x = LeakyReLU(alpha=0.2)(x)
    for _ in range(4):
        x = ZeroPadding2D()(x)
        x = Conv2D(ndf*2, kernel_size=4, strides=2, use_bias=False, kernel_initializer=conv_init)(x)
        x = BatchNormalization(epsilon=1.01e-5, gamma_init=gamma_init)(x, training=1)
        x = LeakyReLU(alpha=0.2)(x)
        ndf *= 2    
    x = Conv2D(1, kernel_size=3, strides=1, use_bias=False, kernel_initializer=conv_init)(x)
    outputs = Flatten()(x)
    return Model(inputs=inputs, outputs=outputs)

def DCGAN_G(isize, nz, ngf):
    inputs = Input(shape=(nz,))
    x = Reshape((1, 1, nz))(inputs)
    x = Conv2DTranspose(filters=ngf, kernel_size=3, strides=2, use_bias=False,
                           kernel_initializer = conv_init)(x)
    for _ in range(4):
        x = Conv2DTranspose(filters=int(ngf/2), kernel_size=4, strides=2, use_bias=False,
                        kernel_initializer = conv_init)(x)
        x = Cropping2D(cropping=1)(x)
        x = BatchNormalization(epsilon=1.01e-5, gamma_init=gamma_init)(x, training=1) 
        x = Activation("relu")(x)
        ngf = int(ngf/2)
    x = Conv2DTranspose(filters=3, kernel_size=4, strides=2, use_bias=False,
                        kernel_initializer = conv_init)(x)
    x = Cropping2D(cropping=1)(x)
    outputs = Activation("tanh")(x)

    return Model(inputs=inputs, outputs=outputs)

## Run Model

In [3]:
nc = 3
nz = 100
ngf = 1024
ndf = 64
imageSize = 96
batchSize = 64
lrD = 0.00005 
lrG = 0.00005
clamp_lower, clamp_upper = -0.01, 0.01  

netD = DCGAN_D(imageSize, nc, ndf)
netD.summary()

netG = DCGAN_G(imageSize, nz, ngf)
netG.summary()

clamp_updates = [K.update(v, K.clip(v, clamp_lower, clamp_upper))
                          for v in netD.trainable_weights]
netD_clamp = K.function([],[], clamp_updates)

netD_real_input = Input(shape=(imageSize, imageSize, nc))
noisev = Input(shape=(nz,))

loss_real = K.mean(netD(netD_real_input))
loss_fake = K.mean(netD(netG(noisev)))
loss = loss_fake - loss_real 
training_updates = RMSprop(lr=lrD).get_updates(netD.trainable_weights,[], loss)
netD_train = K.function([netD_real_input, noisev],
                        [loss_real, loss_fake],    
                        training_updates)

loss = -loss_fake 
training_updates = RMSprop(lr=lrG).get_updates(netG.trainable_weights,[], loss)
netG_train = K.function([noisev], [loss], training_updates)

fixed_noise = np.random.normal(size=(batchSize, nz)).astype('float32')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 98, 98, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        3072      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 64)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       131072    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       512       
__________

### Generate

In [ ]:
datagen = ImageDataGenerator(rotation_range=20,rescale=1./255)

train_generate = datagen.flow_from_directory("faces/", target_size=(96,96), batch_size=64, 
                                                shuffle=True, class_mode=None, save_format='jpg')

step = 0
for step in range(1000):   
    
    for _ in range(5):
        real_data = (np.array(train_generate.next())*2-1)
        noise = np.random.normal(size=(batchSize, nz))
        errD_real, errD_fake  = netD_train([real_data, noise])
        errD = errD_real - errD_fake
        netD_clamp([])
    
    noise = np.random.normal(size=(batchSize, nz))  
    errG, = netG_train([noise])    
    print('[%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f' % (step, errD, errG, errD_real, errD_fake))
            
    if step%1000==0:
        netD.save("discriminator.h5")
        netG.save("generate.h5")
        fake = netG.predict(fixed_noise)
        display_grid = np.zeros((8*96,8*96,3))
        
        for j in range(int(64/8)):
            for k in range(int(64/8)):
                display_grid[j*96:(j+1)*96,k*96:(k+1)*96,:] = fake[k+8*j]
        img_save_path = os.path.join(os.getcwd(),"saved/img/{}.png".format(step))
        scipy.misc.imsave(img_save_path, display_grid)


Found 70171 images belonging to 1 classes.
[0] Loss_D: 0.566005 Loss_G: 0.256068 Loss_D_real: 0.322569 Loss_D_fake -0.243437
[1] Loss_D: 0.660386 Loss_G: 0.319584 Loss_D_real: 0.362268 Loss_D_fake -0.298118
[2] Loss_D: 0.733436 Loss_G: 0.366352 Loss_D_real: 0.378379 Loss_D_fake -0.355056
[3] Loss_D: 0.817330 Loss_G: 0.396930 Loss_D_real: 0.420806 Loss_D_fake -0.396523
[4] Loss_D: 0.896370 Loss_G: 0.437850 Loss_D_real: 0.468237 Loss_D_fake -0.428134
[5] Loss_D: 0.959617 Loss_G: 0.471052 Loss_D_real: 0.502239 Loss_D_fake -0.457378
[6] Loss_D: 0.997395 Loss_G: 0.492967 Loss_D_real: 0.510041 Loss_D_fake -0.487355
[7] Loss_D: 1.043238 Loss_G: 0.517981 Loss_D_real: 0.533016 Loss_D_fake -0.510222
[8] Loss_D: 1.094961 Loss_G: 0.543756 Loss_D_real: 0.557173 Loss_D_fake -0.537788
[9] Loss_D: 1.131082 Loss_G: 0.565636 Loss_D_real: 0.569888 Loss_D_fake -0.561194
[10] Loss_D: 1.171808 Loss_G: 0.589589 Loss_D_real: 0.588830 Loss_D_fake -0.582978
[11] Loss_D: 1.214115 Loss_G: 0.609754 Loss_D_real: 0.

[99] Loss_D: 1.734670 Loss_G: 0.865051 Loss_D_real: 0.869272 Loss_D_fake -0.865397
[100] Loss_D: 1.733494 Loss_G: 0.865185 Loss_D_real: 0.869208 Loss_D_fake -0.864286
[101] Loss_D: 1.730089 Loss_G: 0.864863 Loss_D_real: 0.867185 Loss_D_fake -0.862903
[102] Loss_D: 1.735545 Loss_G: 0.865835 Loss_D_real: 0.869711 Loss_D_fake -0.865835
[103] Loss_D: 1.729411 Loss_G: 0.864182 Loss_D_real: 0.867457 Loss_D_fake -0.861954
[104] Loss_D: 1.734959 Loss_G: 0.866022 Loss_D_real: 0.869195 Loss_D_fake -0.865764
[105] Loss_D: 1.732935 Loss_G: 0.863775 Loss_D_real: 0.867797 Loss_D_fake -0.865138
[106] Loss_D: 1.731905 Loss_G: 0.865517 Loss_D_real: 0.867250 Loss_D_fake -0.864655
[107] Loss_D: 1.737384 Loss_G: 0.866406 Loss_D_real: 0.871168 Loss_D_fake -0.866215
[108] Loss_D: 1.729859 Loss_G: 0.863055 Loss_D_real: 0.866021 Loss_D_fake -0.863838
[109] Loss_D: 1.735187 Loss_G: 0.865297 Loss_D_real: 0.869704 Loss_D_fake -0.865483
[110] Loss_D: 1.732318 Loss_G: 0.865607 Loss_D_real: 0.867553 Loss_D_fake -0.

[197] Loss_D: 1.687335 Loss_G: 0.848635 Loss_D_real: 0.863892 Loss_D_fake -0.823443
[198] Loss_D: 1.735773 Loss_G: 0.866305 Loss_D_real: 0.869685 Loss_D_fake -0.866088
[199] Loss_D: 1.737064 Loss_G: 0.866883 Loss_D_real: 0.870312 Loss_D_fake -0.866752
[200] Loss_D: 1.735633 Loss_G: 0.866441 Loss_D_real: 0.869194 Loss_D_fake -0.866439
[201] Loss_D: 1.715752 Loss_G: 0.861922 Loss_D_real: 0.861360 Loss_D_fake -0.854392
[202] Loss_D: 1.736203 Loss_G: 0.867017 Loss_D_real: 0.869780 Loss_D_fake -0.866423
[203] Loss_D: 1.737774 Loss_G: 0.867166 Loss_D_real: 0.870628 Loss_D_fake -0.867146
[204] Loss_D: 1.717441 Loss_G: 0.840878 Loss_D_real: 0.860467 Loss_D_fake -0.856974
[205] Loss_D: 1.730778 Loss_G: 0.865450 Loss_D_real: 0.868377 Loss_D_fake -0.862401
[206] Loss_D: 1.736160 Loss_G: 0.866985 Loss_D_real: 0.869312 Loss_D_fake -0.866848
[207] Loss_D: 1.737439 Loss_G: 0.866594 Loss_D_real: 0.870728 Loss_D_fake -0.866710
[208] Loss_D: 1.726903 Loss_G: 0.865071 Loss_D_real: 0.864718 Loss_D_fake -0

[295] Loss_D: 1.728957 Loss_G: 0.868518 Loss_D_real: 0.863920 Loss_D_fake -0.865037
[296] Loss_D: 1.741934 Loss_G: 0.869189 Loss_D_real: 0.872644 Loss_D_fake -0.869290
[297] Loss_D: 1.742521 Loss_G: 0.869137 Loss_D_real: 0.873314 Loss_D_fake -0.869207
[298] Loss_D: 1.713539 Loss_G: 0.862383 Loss_D_real: 0.858061 Loss_D_fake -0.855478
[299] Loss_D: 1.739899 Loss_G: 0.867993 Loss_D_real: 0.872587 Loss_D_fake -0.867312
[300] Loss_D: 1.741931 Loss_G: 0.868738 Loss_D_real: 0.873121 Loss_D_fake -0.868811
[301] Loss_D: 1.738446 Loss_G: 0.861348 Loss_D_real: 0.872100 Loss_D_fake -0.866346
[302] Loss_D: 1.739583 Loss_G: 0.868736 Loss_D_real: 0.871889 Loss_D_fake -0.867694
[303] Loss_D: 1.741659 Loss_G: 0.868268 Loss_D_real: 0.873097 Loss_D_fake -0.868563
[304] Loss_D: 1.736538 Loss_G: 0.868812 Loss_D_real: 0.871163 Loss_D_fake -0.865376
[305] Loss_D: 1.741975 Loss_G: 0.869076 Loss_D_real: 0.872914 Loss_D_fake -0.869061
[306] Loss_D: 1.742213 Loss_G: 0.868938 Loss_D_real: 0.873271 Loss_D_fake -0

[393] Loss_D: 1.735540 Loss_G: 0.867440 Loss_D_real: 0.867645 Loss_D_fake -0.867896
[394] Loss_D: 1.738189 Loss_G: 0.868535 Loss_D_real: 0.870571 Loss_D_fake -0.867617
[395] Loss_D: 1.737254 Loss_G: 0.864736 Loss_D_real: 0.870375 Loss_D_fake -0.866879
[396] Loss_D: 1.735164 Loss_G: 0.867731 Loss_D_real: 0.867890 Loss_D_fake -0.867274
[397] Loss_D: 1.741131 Loss_G: 0.868588 Loss_D_real: 0.872396 Loss_D_fake -0.868736
[398] Loss_D: 1.739199 Loss_G: 0.867850 Loss_D_real: 0.871820 Loss_D_fake -0.867379
[399] Loss_D: 1.731528 Loss_G: 0.865713 Loss_D_real: 0.868315 Loss_D_fake -0.863212
[400] Loss_D: 1.738144 Loss_G: 0.868678 Loss_D_real: 0.869500 Loss_D_fake -0.868644
[401] Loss_D: 1.740774 Loss_G: 0.868893 Loss_D_real: 0.872274 Loss_D_fake -0.868500
[402] Loss_D: 1.739544 Loss_G: 0.867333 Loss_D_real: 0.871291 Loss_D_fake -0.868253
[403] Loss_D: 1.729867 Loss_G: 0.868083 Loss_D_real: 0.866119 Loss_D_fake -0.863747
[404] Loss_D: 1.735223 Loss_G: 0.866692 Loss_D_real: 0.869700 Loss_D_fake -0

[491] Loss_D: 1.730374 Loss_G: 0.864143 Loss_D_real: 0.865930 Loss_D_fake -0.864443
[492] Loss_D: 1.712760 Loss_G: 0.850301 Loss_D_real: 0.852520 Loss_D_fake -0.860240
[493] Loss_D: 1.724717 Loss_G: 0.863124 Loss_D_real: 0.866349 Loss_D_fake -0.858368
[494] Loss_D: 1.728065 Loss_G: 0.863111 Loss_D_real: 0.867858 Loss_D_fake -0.860206
[495] Loss_D: 1.723884 Loss_G: 0.857929 Loss_D_real: 0.861733 Loss_D_fake -0.862151
[496] Loss_D: 1.718124 Loss_G: 0.853346 Loss_D_real: 0.868271 Loss_D_fake -0.849853
[497] Loss_D: 1.730930 Loss_G: 0.864764 Loss_D_real: 0.865370 Loss_D_fake -0.865560
[498] Loss_D: 1.725872 Loss_G: 0.858837 Loss_D_real: 0.865056 Loss_D_fake -0.860816
[499] Loss_D: 1.729718 Loss_G: 0.864228 Loss_D_real: 0.865561 Loss_D_fake -0.864157
[500] Loss_D: 1.721761 Loss_G: 0.861585 Loss_D_real: 0.867444 Loss_D_fake -0.854317
[501] Loss_D: 1.728921 Loss_G: 0.862232 Loss_D_real: 0.865082 Loss_D_fake -0.863839
[502] Loss_D: 1.734889 Loss_G: 0.866839 Loss_D_real: 0.868272 Loss_D_fake -0

[589] Loss_D: 1.721425 Loss_G: 0.860029 Loss_D_real: 0.861801 Loss_D_fake -0.859623
[590] Loss_D: 1.719417 Loss_G: 0.860704 Loss_D_real: 0.861486 Loss_D_fake -0.857931
[591] Loss_D: 1.717081 Loss_G: 0.858191 Loss_D_real: 0.861126 Loss_D_fake -0.855955
[592] Loss_D: 1.714244 Loss_G: 0.857631 Loss_D_real: 0.854815 Loss_D_fake -0.859429
[593] Loss_D: 1.717283 Loss_G: 0.860431 Loss_D_real: 0.859778 Loss_D_fake -0.857505
[594] Loss_D: 1.679692 Loss_G: 0.845889 Loss_D_real: 0.830064 Loss_D_fake -0.849628
[595] Loss_D: 0.671355 Loss_G: 0.852666 Loss_D_real: 0.844430 Loss_D_fake 0.173075
[596] Loss_D: 1.690120 Loss_G: 0.855079 Loss_D_real: 0.840617 Loss_D_fake -0.849503
[597] Loss_D: 1.697783 Loss_G: 0.851629 Loss_D_real: 0.844711 Loss_D_fake -0.853073
[598] Loss_D: 1.708852 Loss_G: 0.855775 Loss_D_real: 0.855986 Loss_D_fake -0.852866
[599] Loss_D: 1.711303 Loss_G: 0.855255 Loss_D_real: 0.854570 Loss_D_fake -0.856734
[600] Loss_D: 1.711229 Loss_G: 0.857386 Loss_D_real: 0.857914 Loss_D_fake -0.

[687] Loss_D: 1.707445 Loss_G: 0.854939 Loss_D_real: 0.851384 Loss_D_fake -0.856060
[688] Loss_D: 1.698839 Loss_G: 0.849888 Loss_D_real: 0.854343 Loss_D_fake -0.844496
[689] Loss_D: 1.697032 Loss_G: 0.861342 Loss_D_real: 0.846918 Loss_D_fake -0.850115
[690] Loss_D: 1.718252 Loss_G: 0.860614 Loss_D_real: 0.858940 Loss_D_fake -0.859312
[691] Loss_D: 1.712932 Loss_G: 0.857781 Loss_D_real: 0.858184 Loss_D_fake -0.854748
[692] Loss_D: 1.708486 Loss_G: 0.851920 Loss_D_real: 0.859725 Loss_D_fake -0.848761
[693] Loss_D: 0.744920 Loss_G: 0.846044 Loss_D_real: 0.865692 Loss_D_fake 0.120771
[694] Loss_D: 1.681370 Loss_G: 0.851613 Loss_D_real: 0.833241 Loss_D_fake -0.848129
[695] Loss_D: 1.703574 Loss_G: 0.854367 Loss_D_real: 0.848932 Loss_D_fake -0.854642
[696] Loss_D: 1.711145 Loss_G: 0.857108 Loss_D_real: 0.855180 Loss_D_fake -0.855965
[697] Loss_D: 1.709433 Loss_G: 0.857741 Loss_D_real: 0.851833 Loss_D_fake -0.857601
[698] Loss_D: 1.720059 Loss_G: 0.859672 Loss_D_real: 0.859539 Loss_D_fake -0.

[785] Loss_D: 1.724159 Loss_G: 0.861393 Loss_D_real: 0.862727 Loss_D_fake -0.861432
[786] Loss_D: 1.715759 Loss_G: 0.854054 Loss_D_real: 0.862341 Loss_D_fake -0.853418
[787] Loss_D: 1.715883 Loss_G: 0.859566 Loss_D_real: 0.860945 Loss_D_fake -0.854938
[788] Loss_D: 1.715722 Loss_G: 0.856372 Loss_D_real: 0.860317 Loss_D_fake -0.855405
[789] Loss_D: 1.716999 Loss_G: 0.858296 Loss_D_real: 0.857261 Loss_D_fake -0.859738
[790] Loss_D: 1.712667 Loss_G: 0.859097 Loss_D_real: 0.856485 Loss_D_fake -0.856182
[791] Loss_D: 1.717118 Loss_G: 0.857764 Loss_D_real: 0.862031 Loss_D_fake -0.855087
[792] Loss_D: 0.235852 Loss_G: 0.758795 Loss_D_real: -0.619459 Loss_D_fake -0.855311
[793] Loss_D: 1.711845 Loss_G: 0.860697 Loss_D_real: 0.854287 Loss_D_fake -0.857558
[794] Loss_D: 1.718629 Loss_G: 0.860051 Loss_D_real: 0.858038 Loss_D_fake -0.860590
[795] Loss_D: 1.720699 Loss_G: 0.861612 Loss_D_real: 0.860704 Loss_D_fake -0.859995
[796] Loss_D: 1.720002 Loss_G: 0.859359 Loss_D_real: 0.860036 Loss_D_fake -

[883] Loss_D: 1.706106 Loss_G: 0.852123 Loss_D_real: 0.851181 Loss_D_fake -0.854925
[884] Loss_D: 0.706175 Loss_G: 0.830023 Loss_D_real: 0.859634 Loss_D_fake 0.153459
[885] Loss_D: 1.671213 Loss_G: 0.852630 Loss_D_real: 0.836043 Loss_D_fake -0.835170
[886] Loss_D: 1.696107 Loss_G: 0.851548 Loss_D_real: 0.846251 Loss_D_fake -0.849856
[887] Loss_D: 1.692136 Loss_G: 0.850696 Loss_D_real: 0.843220 Loss_D_fake -0.848916
[888] Loss_D: 1.684289 Loss_G: 0.842107 Loss_D_real: 0.838522 Loss_D_fake -0.845767
[889] Loss_D: 1.518880 Loss_G: 0.841422 Loss_D_real: 0.774087 Loss_D_fake -0.744793
[890] Loss_D: 1.666644 Loss_G: 0.843149 Loss_D_real: 0.820357 Loss_D_fake -0.846287
[891] Loss_D: 1.683568 Loss_G: 0.849855 Loss_D_real: 0.840991 Loss_D_fake -0.842577
[892] Loss_D: 1.686764 Loss_G: 0.845678 Loss_D_real: 0.838216 Loss_D_fake -0.848548
[893] Loss_D: 1.697202 Loss_G: 0.852641 Loss_D_real: 0.850400 Loss_D_fake -0.846802
